# 🔧 Model Configuration Manager for RAG Analysis

This notebook allows you to create, manage, and select between different **LLM model configurations** for use with tools like `ChatGPTAnalyzer`.

Each configuration includes:
- 🤖 **Model name** (e.g., `gpt-4o`, `mistral-7b-instruct`, `llama3-70b-instruct`)
- 🌐 **Base URL** (e.g., `https://api.openai.com/v1`, `http://localhost:8080/v1`)
- 🔐 **API Key** (e.g., `sk-...` or `ollama`, `sk-noauth`)

---

## 🗃️ Configuration Storage

All configurations are stored in the following JSON file:

```
~/.secrets/model_configs.json
```

This file will look something like:

```json
{
  "openai": {
    "model": "gpt-4o",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-..."
  },
  "mistral": {
    "model": "mistral-7b-instruct",
    "base_url": "http://localhost:8080/v1",
    "api_key": "sk-noauth"
  },
  "_default": "openai"
}
```

You can **manually edit or copy** this file if needed to:
- Share configurations with others
- Maintain consistent setups across environments
- Set or change the default model (`"_default"` key)

---

## ✅ How to Use This Notebook

1. Run the notebook cells to:
   - Add or update a named model configuration
   - Optionally set it as the default

2. Use your configurations in tools like `ChatGPTAnalyzer`:

```python
from Open_AI_RAG_manager import ChatGPTAnalyzer

# Use specific model
analyzer = ChatGPTAnalyzer.from_config("mistral", yaml_content=my_yaml)

# Or just use the default model
analyzer = ChatGPTAnalyzer.from_config(yaml_content=my_yaml)
```

---

> 💡 Tip: You can define as many configurations as you like — great for testing different LLMs or backends like OpenAI, LM Studio, Ollama, and Hugging Face APIs.



In [1]:
import json
from pathlib import Path
from getpass import getpass
from IPython.display import display, Markdown

CONFIG_FILE = Path.home() / ".secrets" / "model_configs.json"
CONFIG_FILE.parent.mkdir(exist_ok=True)

def load_configs():
    if CONFIG_FILE.exists():
        with CONFIG_FILE.open() as f:
            return json.load(f)
    return {}

def save_configs(configs):
    with CONFIG_FILE.open("w") as f:
        json.dump(configs, f, indent=2)

def create_or_update_config():
    name = input("🔖 Enter configuration name (e.g., mistral): ").strip()
    model = input("🤖 Model name (e.g., mistral-7b-instruct): ").strip()
    base_url = input("🌐 Base URL (e.g., http://localhost:8080/v1): ").strip()
    api_key = getpass("🔐 API Key (or sk-noauth): ").strip()
    
    configs = load_configs()
    configs[name] = {
        "model": model,
        "base_url": base_url,
        "api_key": api_key
    }

    # Set as default?
    make_default = input("⭐ Set this config as default? (y/N): ").strip().lower() == "y"
    if make_default:
        configs["_default"] = name

    save_configs(configs)
    display(Markdown(f"✅ **Configuration `{name}` saved.**"))
    if make_default:
        display(Markdown(f"🌟 **Set as default configuration.**"))

def list_configs():
    configs = load_configs()
    if not configs:
        display(Markdown("❌ **No configurations found.**"))
        return

    default_name = configs.get("_default", "None")
    display(Markdown(f"### 📦 Saved Configurations (default: `{default_name}`)"))
    for name, config in configs.items():
        if name == "_default":
            continue
        display(Markdown(f"- `{name}` → model: `{config['model']}`, url: `{config['base_url']}`"))


list_configs()
create_or_update_config()
list_configs()


### 📦 Saved Configurations (default: `gpt-oss-120b`)

- `gpt-oss-120b` → model: `gpt-oss-120b`, url: `https://api.siemens.com/llm`

🔖 Enter configuration name (e.g., mistral):  gpt-4o
🤖 Model name (e.g., mistral-7b-instruct):  gpt-4o
🌐 Base URL (e.g., http://localhost:8080/v1):  https://api.openai.com/v1
🔐 API Key (or sk-noauth):  ········
⭐ Set this config as default? (y/N):  y


✅ **Configuration `gpt-4o` saved.**

🌟 **Set as default configuration.**

### 📦 Saved Configurations (default: `gpt-4o`)

- `gpt-oss-120b` → model: `gpt-oss-120b`, url: `https://api.siemens.com/llm`

- `gpt-4o` → model: `gpt-4o`, url: `https://api.openai.com/v1`